In [3]:
import backtrader as bt
import yfinance as yf
import pandas as pd

In [20]:
class LongStraddleSimplified(bt.Strategy):
    params = (
        ('atr_period', 14),
        ('atr_multiplier', 1.0),
        ('atr_sma_period', 20),
        ('sma_period', 20),
        # ('stop_loss_percent', 2.0),
        ('rrr', 2.0),  # TP is 2x the risk distance
    )

    def __init__(self):
        self.atr = bt.indicators.ATR(period=self.params.atr_period)
        self.atr_sma = bt.indicators.SMA(self.atr, period=self.params.atr_sma_period)
        self.close_sma = bt.indicators.SMA(self.datas[0].close, period=self.params.sma_period)

        # To prevent placing multiple orders while one is pending
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                print(
                    f'{self.datas[0].datetime.date(0)} BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, '
                    f'Size: {order.executed.size}'
                )

            elif order.issell():
                print(
                    f'{self.datas[0].datetime.date(0)} SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, '
                    f'Size: {order.executed.size}'
                )

            self.bar_executed = len(self) # Bar number when order was executed

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f'{self.datas[0].datetime.date(0)} Order Canceled/Margin/Rejected')

        # Reset order status tracking
        self.order = None

    def notify_trade(self, trade):
         if not trade.isclosed:
             return
         print(f'{self.datas[0].datetime.date(0)} TRADE PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')


    def next(self):
        # Wait for indicators to have enough data
        # Use max period from all indicators used in the condition + 1
        if len(self) <= max(self.params.atr_period, 20) + 1:
             return

        # Check if an order is pending. If so, do not send another one
        # if self.order:
        #     return

        # Check if we are already in the market
        if self.position:
            return
    
        # Calculate current indicator values
        atr_value = self.atr[0]
        atr_sma_value = self.atr_sma[0]
        close_value = self.datas[0].close[0]
        close_sma_value = self.close_sma[0]

        # Define entry condition
        high_volatility_condition = close_value > close_sma_value and atr_value > atr_sma_value

        if high_volatility_condition:
            # Calculate Stop Loss Price based on current close (potential entry)
            potential_entry = close_value
            # stop_loss_price = potential_entry * (1.0 - self.params.stop_loss_percent / 100.0)
            # take_profit_price = potential_entry * (1.0 + self.take_profit_percent / 100.0)
            sl_diff = atr_value * self.params.atr_multiplier
            stop_loss_price = potential_entry - sl_diff
            take_profit_price = potential_entry + self.params.rrr * sl_diff
            print(f"{self.datas[0].datetime.date(0)} --- ENTRY SIGNAL ---")
            print(f"Potential Entry: {potential_entry:.2f}")
            print(f"Stop Loss Price: {stop_loss_price:.2f}")
            print(f"Take Profit Price: {take_profit_price:.2f}")

            # Place Buy order with attached Stop Loss and Take Profit (Bracket Order)
            self.order = self.buy_bracket(
                exectype=bt.Order.Market,
                transmit=True, # Transmit the main order immediately
                # The following create virtual child orders managed by backtrader
                stopprice=stop_loss_price,
                stopexec=bt.Order.Stop, # Stop Market order for SL
                limitprice=take_profit_price,
                limitexec=bt.Order.Limit # Limit order for TP
            )
            # The old way using parent/transmit=False is more complex
            # self.mainside = self.buy(size=size, exectype=bt.Order.Market)
            # self.sell(exectype=bt.Order.Stop, price=stop_loss_price, size=size, parent=self.mainside)
            # self.sell(exectype=bt.Order.Limit, price=take_profit_price, size=size, parent=self.mainside)


In [33]:
# Fetch S&P500 data from Yahoo Finance
# ticker = "^GSPC"  # S&P 500 ticker
# ticker = "AAPL"  # Apple ticker

# ticker = "USDJPY=X"  # USD/JPY ticker
# ticker = "^DJI"  # Dow Jones Industrial Average ticker
# ticker = "^IXIC"  # NASDAQ Composite ticker
# Download Tesla data
ticker = "TSLA"  # Tesla ticker
# Download data from Yahoo Finance
data = yf.download(ticker, start="2018-01-01", end="2025-10-01", interval="1d", multi_level_index=False)

# Convert to Backtrader data feed
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# Clean columns to match Backtrader's expectations
# data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
# data.columns = ['open', 'high', 'low', 'close', 'volume']
# data['openinterest'] = 0  # Backtrader requires this column
# data['close'] = data['close'].astype(float)
# data['open'] = data['open'].astype(float)
# data['high'] = data['high'].astype(float)
# data['low'] = data['low'].astype(float)
# data['volume'] = data['volume'].astype(float)
# data['openinterest'] = data['openinterest'].astype(float)
# data['datetime'] = data.index
# data['datetime'] = pd.to_datetime(data['datetime'])
# data['datetime'] = data['datetime'].apply(lambda x: x.timestamp())
# data['datetime'] = pd.to_datetime(data['datetime'], unit='s')
# data['datetime'] = data['datetime'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

data_feed = bt.feeds.PandasData(dataname=data,
                                datetime=0, open=1, high=2, low=3, close=4, volume=5, openinterest=-1
                                )

[*********************100%***********************]  1 of 1 completed


In [34]:
cerebro = bt.Cerebro()

# Add Strategy
cerebro.addstrategy(LongStraddleSimplified, atr_period=14, rrr=1.5, atr_multiplier=1.5)

cerebro.adddata(data_feed)
cerebro.addsizer(bt.sizers.PercentSizer, percents=2)

# Set initial capital
cerebro.broker.setcash(100000.0)

# Set commission - Simplified for example, adjust as needed
cerebro.broker.setcommission(commission=0.001)

# Analyzer for performance metrics
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

In [35]:
# Run Backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value:   %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2018-02-22 --- ENTRY SIGNAL ---
Potential Entry: 22.37
Stop Loss Price: 20.89
Take Profit Price: 24.59
2018-02-23 BUY EXECUTED, Price: 23.47, Cost: 2098.47, Comm: 2.10, Size: 89.41077919960598
2018-03-19 SELL EXECUTED, Price: 20.89, Cost: 2098.47, Comm: 1.87, Size: -89.41077919960598
2018-03-19 Order Canceled/Margin/Rejected
2018-03-19 TRADE PROFIT, GROSS -231.08, NET -235.05
2018-04-11 --- ENTRY SIGNAL ---
Potential Entry: 20.05
Stop Loss Price: 18.02
Take Profit Price: 23.09
2018-04-12 BUY EXECUTED, Price: 19.61, Cost: 1951.11, Comm: 1.95, Size: 99.51947229994434
2018-06-12 SELL EXECUTED, Price: 23.09, Cost: 1951.11, Comm: 2.30, Size: -99.51947229994434
2018-06-12 Order Canceled/Margin/Rejected
2018-06-12 TRADE PROFIT, GROSS 346.68, NET 342.43
2018-06-12 --- ENTRY SIGNAL ---
Potential Entry: 22.98
Stop Loss Price: 21.47
Take Profit Price: 25.25
2018-06-13 BUY EXECUTED, Price: 22.99, Cost: 2002.61, Comm: 2.00, Size: 87.12566216877194
2018-07-03 SELL

In [36]:
# Extract Analyzers
returns_analyzer = results[0].analyzers.returns.get_analysis()
sharpe_analyzer = results[0].analyzers.sharpe.get_analysis()
drawdown_analyzer = results[0].analyzers.drawdown.get_analysis()

print('Returns:', returns_analyzer['rtot'])
print('Sharpe Ratio:', sharpe_analyzer['sharperatio'])
print('Drawdown:', drawdown_analyzer['drawdown'])
print('Max Drawdown:', drawdown_analyzer['max']['drawdown'])
# Optional: Plot the backtest result
cerebro.plot()

Returns: 0.038055530551116455
Sharpe Ratio: 0.3467378812142013
Drawdown: 0.8252883689856237
Max Drawdown: 1.1742764186997474


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]